In [1]:
%autosave 0

Autosave disabled


In [2]:
import numpy as np
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
test.dtypes['Id']

dtype('int64')

In [4]:
np.dtype('object')

dtype('O')

In [5]:
num_columns = []
cate_columns = []
for column in test.columns:
    if test.dtypes[column] != np.dtype('object'):
        num_columns.append(column)
    else:
        cate_columns.append(column)

In [6]:
num_columns

['Id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [7]:
cate_columns

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [6]:
label = train.pop('SalePrice')

In [7]:
for column in num_columns:
    train[column] = train[column].fillna(train[column].median())
    test[column] = test[column].fillna(test[column].median())
for column in cate_columns:
    train[column] = train[column].fillna(train[column].mode())
    test[column] = test[column].fillna(test[column].mode())
# for column in cate_columns:
#     train[column] = train[column].fillna('NaN')
#     test[column] = test[column].fillna('NaN')

In [8]:
data = pd.concat([train, test], axis=0)
for column in cate_columns:
    t = pd.get_dummies(data[column], prefix=column)
    
    train = pd.concat([train, t[:len(train)]], axis=1)
    train.drop(column, axis=1, inplace=True)
    
    test = pd.concat([test, t[len(train):]], axis=1)
    test.drop(column, axis=1, inplace=True)

In [10]:
from sklearn.linear_model import Lasso, LinearRegression, Ridge, ElasticNet, TheilSenRegressor, HuberRegressor, RANSACRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
# from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import itertools

In [11]:
X_train, X_test, y_train, y_test = train_test_split(train, label, test_size=0.33, random_state=42)

In [12]:
regs = [['Lasso', Lasso()],
       ['LinearRegression', LinearRegression()],
       ['Ridge', Ridge()],
       ['ElasticNet', ElasticNet()],
       ['TheilSenRegressor', TheilSenRegressor()],
       ['RANSACRegressor', RANSACRegressor()],
       ['HuberRegressor', HuberRegressor()],
       ['SVR', SVR(kernel='linear')],
       ['DecisionTreeRegressor', DecisionTreeRegressor()],
       ['ExtraTreeRegressor', ExtraTreeRegressor()],
       ['AdaBoostRegressor', AdaBoostRegressor(n_estimators=150)],
       ['ExtraTreesRegressor', ExtraTreesRegressor(n_estimators=150)],
       ['GradientBoostingRegressor', GradientBoostingRegressor(n_estimators=150)],
       ['RandomForestRegressor', RandomForestRegressor(n_estimators=150)]]
#        ['XGBRegressor', XGBRegressor(n_estimators=150)]]

In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 978 entries, 615 to 1126
Columns: 289 entries, Id to SaleCondition_Partial
dtypes: float64(3), int64(34), uint8(252)
memory usage: 531.0 KB


In [17]:
y_train

615     137500
613     147000
1303    232000
486     156000
561     170000
308      82500
461     155000
1142    424870
730     236500
1155    218000
1203    213000
700     312500
849     187000
1260    181000
787     233000
352      95000
710      52000
124     181000
178     501837
287      88000
1407    112000
1208    140000
294     167000
327     145250
1456    210000
841     157500
1121    212900
931     117500
236     185500
88       85000
         ...  
747     265979
252     173000
21      139400
1337     52500
459     110000
1184    186700
276     201000
955     145000
1215    125000
385     192000
805     227680
1437    394617
343     266000
769     538000
1332    100000
130     226000
871     200500
1123    118000
1396    160000
87      164500
330     119000
1238    142500
466     167000
121     100000
1044    278000
1095    176432
1130    135000
1294    115000
860     189950
1126    174000
Name: SalePrice, dtype: int64

In [18]:
X_train

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
615,616,85,80.0,8800,6,7,1963,1963,156.0,763,...,0,0,0,1,1,0,0,0,0,0
613,614,20,70.0,8402,5,5,2007,2007,0.0,206,...,0,1,0,0,0,0,0,0,0,1
1303,1304,20,73.0,8688,7,5,2005,2005,228.0,0,...,0,0,0,1,0,0,0,0,1,0
486,487,20,79.0,10289,5,7,1965,1965,168.0,836,...,0,0,0,1,0,0,0,0,1,0
561,562,20,77.0,10010,5,5,1974,1975,0.0,1071,...,0,0,0,1,0,0,0,0,1,0
308,309,30,NaN,12342,4,5,1940,1950,0.0,262,...,0,0,0,1,0,0,0,0,1,0
461,462,70,60.0,7200,7,9,1936,2007,0.0,350,...,0,0,0,1,0,0,0,0,1,0
1142,1143,60,77.0,9965,8,5,2006,2007,340.0,1150,...,0,1,0,0,0,0,0,0,0,1
730,731,120,39.0,5389,8,5,1995,1996,0.0,1180,...,0,0,0,1,0,0,0,0,1,0
1155,1156,20,90.0,10768,5,8,1976,2004,0.0,1157,...,0,0,0,1,0,0,0,0,1,0


In [13]:
preds = []
for reg_name, reg in regs:
    print(reg_name)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    if np.sum(y_pred < 0) > 0:
        print('y_pred have ', np.sum(y_pred < 0), 'negative values, we fill it np.median(y_pred)')
        y_pred[y_pred<0] = np.median(y_pred)
    score = np.sqrt(mean_squared_error(np.log(y_test), np.log(y_pred)))
    preds.append([reg_name, y_pred])

Lasso


/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


LinearRegression
Ridge
ElasticNet
TheilSenRegressor
y_pred have  1 negative values, we fill it np.median(y_pred)
RANSACRegressor
HuberRegressor
y_pred have  1 negative values, we fill it np.median(y_pred)
SVR
DecisionTreeRegressor
ExtraTreeRegressor
AdaBoostRegressor
ExtraTreesRegressor
GradientBoostingRegressor
RandomForestRegressor


In [14]:
final_results = []
for comb_length in range(1,len(regs)+1):
    print('Model Amount: ', comb_length)
    results = []
    for comb in itertools.combinations(preds, comb_length):
        pred_sum = 0
        model_name = []
        for reg_name, pred in comb:
            pred_sum += pred
            model_name.append(reg_name)
        pred_sum /= comb_length
        model_name = '+'.join(model_name)
        score = np.sqrt(mean_squared_error(np.log(y_test), np.log(pred_sum)))
        results.append([model_name, score])
    results = sorted(results, key=lambda x: x[1])
    for model_name, score in results:
        print(model_name, score)
    print()
    final_results.append(results[0])

Model Amount:  1
GradientBoostingRegressor 0.13482583710256008
RandomForestRegressor 0.14679837057777684
ExtraTreesRegressor 0.16173194525845586
Ridge 0.1644182688830052
Lasso 0.16495910130882413
ElasticNet 0.17454697125759946
LinearRegression 0.179679992692121
SVR 0.19184149110377383
AdaBoostRegressor 0.20633714514119425
RANSACRegressor 0.20698092009507302
HuberRegressor 0.20708813782380844
TheilSenRegressor 0.2115439654755405
DecisionTreeRegressor 0.21769958948228588
ExtraTreeRegressor 0.25004926020495993

Model Amount:  2
Lasso+GradientBoostingRegressor 0.12826346320121035
LinearRegression+GradientBoostingRegressor 0.13111405800739379
Lasso+RandomForestRegressor 0.13230592217213782
RANSACRegressor+GradientBoostingRegressor 0.13315628512472327
Ridge+GradientBoostingRegressor 0.13321810518032462
LinearRegression+RandomForestRegressor 0.13517758633256352
Ridge+RandomForestRegressor 0.13593191315682718
RANSACRegressor+RandomForestRegressor 0.1369211850311248
GradientBoostingRegressor+Ra

Lasso+RANSACRegressor+GradientBoostingRegressor+RandomForestRegressor 0.1275451408100603
Ridge+RANSACRegressor+GradientBoostingRegressor+RandomForestRegressor 0.12834289136043728
LinearRegression+RANSACRegressor+GradientBoostingRegressor+RandomForestRegressor 0.12873609857303203
Lasso+Ridge+GradientBoostingRegressor+RandomForestRegressor 0.12941706036354628
Lasso+RANSACRegressor+ExtraTreesRegressor+GradientBoostingRegressor 0.12956090884628285
Lasso+LinearRegression+GradientBoostingRegressor+RandomForestRegressor 0.12991716955575308
LinearRegression+Ridge+GradientBoostingRegressor+RandomForestRegressor 0.1302828555267014
Lasso+RANSACRegressor+SVR+GradientBoostingRegressor 0.13042186374750858
Lasso+SVR+GradientBoostingRegressor+RandomForestRegressor 0.13061148281306148
LinearRegression+RANSACRegressor+ExtraTreesRegressor+GradientBoostingRegressor 0.1306323968883161
Ridge+RANSACRegressor+SVR+GradientBoostingRegressor 0.13081460633534758
Lasso+ElasticNet+GradientBoostingRegressor+RandomFo

Lasso+RANSACRegressor+SVR+GradientBoostingRegressor+RandomForestRegressor 0.12816077345912025
Lasso+ElasticNet+RANSACRegressor+GradientBoostingRegressor+RandomForestRegressor 0.128710696414847
Ridge+RANSACRegressor+SVR+GradientBoostingRegressor+RandomForestRegressor 0.12882007598770306
Lasso+Ridge+RANSACRegressor+GradientBoostingRegressor+RandomForestRegressor 0.12896440758329183
Lasso+RANSACRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor 0.12900479713760438
LinearRegression+RANSACRegressor+SVR+GradientBoostingRegressor+RandomForestRegressor 0.1291227419732953
Lasso+RANSACRegressor+SVR+ExtraTreesRegressor+GradientBoostingRegressor 0.12923229387436744
LinearRegression+ElasticNet+RANSACRegressor+GradientBoostingRegressor+RandomForestRegressor 0.12939503243366643
LinearRegression+RANSACRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor 0.12963796449034964
Lasso+Ridge+SVR+GradientBoostingRegressor+RandomForestRegressor 0.129776166308

LinearRegression+HuberRegressor+DecisionTreeRegressor+ExtraTreeRegressor+RandomForestRegressor 0.14824888283960957
Lasso+DecisionTreeRegressor+ExtraTreeRegressor+AdaBoostRegressor+GradientBoostingRegressor 0.1482550977905421
Lasso+ElasticNet+HuberRegressor+ExtraTreeRegressor+ExtraTreesRegressor 0.1482766419032619
ElasticNet+HuberRegressor+SVR+GradientBoostingRegressor+RandomForestRegressor 0.14828832892851737
LinearRegression+ElasticNet+TheilSenRegressor+HuberRegressor+ExtraTreeRegressor 0.14829971702334155
Ridge+DecisionTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+RandomForestRegressor 0.14830069868853762
SVR+DecisionTreeRegressor+ExtraTreeRegressor+GradientBoostingRegressor+RandomForestRegressor 0.14830417214830136
TheilSenRegressor+RANSACRegressor+HuberRegressor+SVR+ExtraTreeRegressor 0.14832360260029268
LinearRegression+ElasticNet+HuberRegressor+SVR+DecisionTreeRegressor 0.14832536776844135
LinearRegression+Ridge+HuberRegressor+ExtraTreeRegressor+AdaBoostRegressor 0.14832720

Lasso+Ridge+RANSACRegressor+SVR+GradientBoostingRegressor+RandomForestRegressor 0.1283307825504489
Lasso+RANSACRegressor+SVR+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor 0.12908379705349599
LinearRegression+Ridge+RANSACRegressor+SVR+GradientBoostingRegressor+RandomForestRegressor 0.12913678381923016
Lasso+LinearRegression+RANSACRegressor+SVR+GradientBoostingRegressor+RandomForestRegressor 0.12923727328776735
Lasso+Ridge+RANSACRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor 0.12930442439358475
Lasso+LinearRegression+RANSACRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor 0.12952310374876508
Lasso+Ridge+RANSACRegressor+SVR+ExtraTreesRegressor+GradientBoostingRegressor 0.12957543827641121
LinearRegression+RANSACRegressor+SVR+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor 0.1296549106070186
Lasso+ElasticNet+RANSACRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor

...

Lasso+Ridge+HuberRegressor+SVR+DecisionTreeRegressor+ExtraTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor 0.1384815149791693
Lasso+LinearRegression+ElasticNet+TheilSenRegressor+HuberRegressor+SVR+DecisionTreeRegressor+RandomForestRegressor 0.13848332367936714
LinearRegression+ElasticNet+TheilSenRegressor+RANSACRegressor+HuberRegressor+DecisionTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor 0.13848693124170652
Ridge+TheilSenRegressor+RANSACRegressor+HuberRegressor+SVR+DecisionTreeRegressor+ExtraTreeRegressor+RandomForestRegressor 0.1384880025330008
Lasso+LinearRegression+ElasticNet+TheilSenRegressor+HuberRegressor+ExtraTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor 0.13848951386424516
Ridge+ElasticNet+RANSACRegressor+SVR+DecisionTreeRegressor+ExtraTreeRegressor+ExtraTreesRegressor+RandomForestRegressor 0.1384925325902621
Lasso+LinearRegression+Ridge+TheilSenRegressor+DecisionTreeRegressor+ExtraTreeRegressor+AdaBoostRegressor+GradientBoostingRegressor 0.13849

Lasso+LinearRegression+Ridge+RANSACRegressor+SVR+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor 0.12946601625560802
Lasso+LinearRegression+Ridge+ElasticNet+RANSACRegressor+SVR+DecisionTreeRegressor+GradientBoostingRegressor+RandomForestRegressor 0.13030315176859522
Lasso+LinearRegression+Ridge+ElasticNet+RANSACRegressor+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor 0.13042310316828007
Lasso+LinearRegression+Ridge+ElasticNet+RANSACRegressor+SVR+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor 0.13043631069345538
Lasso+LinearRegression+ElasticNet+RANSACRegressor+SVR+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor 0.1305521866602062
Lasso+Ridge+TheilSenRegressor+RANSACRegressor+SVR+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor 0.1306134886723124
Lasso+Ridge+ElasticNet+RANSACRegressor+SVR+DecisionTree

Lasso+Ridge+RANSACRegressor+HuberRegressor+SVR+DecisionTreeRegressor+ExtraTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor 0.1389997992107548
Lasso+ElasticNet+TheilSenRegressor+HuberRegressor+SVR+DecisionTreeRegressor+AdaBoostRegressor+GradientBoostingRegressor+RandomForestRegressor 0.1390029599454343
Lasso+LinearRegression+RANSACRegressor+HuberRegressor+SVR+DecisionTreeRegressor+ExtraTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor 0.13901221128336924
Lasso+Ridge+ElasticNet+TheilSenRegressor+SVR+ExtraTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+GradientBoostingRegressor 0.13903592466861628
LinearRegression+Ridge+ElasticNet+TheilSenRegressor+RANSACRegressor+HuberRegressor+DecisionTreeRegressor+ExtraTreeRegressor+AdaBoostRegressor 0.13904096438940824
LinearRegression+Ridge+ElasticNet+TheilSenRegressor+HuberRegressor+ExtraTreeRegressor+AdaBoostRegressor+GradientBoostingRegressor+RandomForestRegressor 0.13904411239807715
Ridge+TheilSenRegressor+RANSACRegressor+HuberRegressor+

Lasso+LinearRegression+Ridge+ElasticNet+RANSACRegressor+SVR+AdaBoostRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor 0.13176556650179333
Lasso+LinearRegression+Ridge+ElasticNet+TheilSenRegressor+RANSACRegressor+SVR+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor 0.1318377952574359
Lasso+LinearRegression+Ridge+TheilSenRegressor+RANSACRegressor+SVR+DecisionTreeRegressor+AdaBoostRegressor+GradientBoostingRegressor+RandomForestRegressor 0.13187098065632835
Lasso+LinearRegression+Ridge+TheilSenRegressor+RANSACRegressor+HuberRegressor+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor 0.13194470889597568
Lasso+LinearRegression+Ridge+ElasticNet+RANSACRegressor+SVR+DecisionTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+GradientBoostingRegressor 0.1320022166842144
Lasso+LinearRegression+Ridge+ElasticNet+TheilSenRegressor+RANSACRegressor+SVR+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegre

Lasso+LinearRegression+Ridge+ElasticNet+RANSACRegressor+HuberRegressor+SVR+ExtraTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+RandomForestRegressor 0.1369702021017105
Lasso+Ridge+ElasticNet+TheilSenRegressor+RANSACRegressor+HuberRegressor+SVR+ExtraTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+GradientBoostingRegressor 0.1369856365642245
LinearRegression+Ridge+ElasticNet+RANSACRegressor+HuberRegressor+SVR+DecisionTreeRegressor+ExtraTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+GradientBoostingRegressor 0.13699135069141938
LinearRegression+Ridge+ElasticNet+TheilSenRegressor+RANSACRegressor+SVR+DecisionTreeRegressor+ExtraTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+RandomForestRegressor 0.1370094507565646
Lasso+LinearRegression+Ridge+ElasticNet+TheilSenRegressor+DecisionTreeRegressor+ExtraTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor 0.13702110367038872
Lasso+LinearRegression+Ridge+ElasticNet+RANSACRegressor+Hu

In [15]:
[b for b in zip(itertools.count(), [a[0] for a in regs])]

[(0, 'Lasso'),
 (1, 'LinearRegression'),
 (2, 'Ridge'),
 (3, 'ElasticNet'),
 (4, 'TheilSenRegressor'),
 (5, 'RANSACRegressor'),
 (6, 'HuberRegressor'),
 (7, 'SVR'),
 (8, 'DecisionTreeRegressor'),
 (9, 'ExtraTreeRegressor'),
 (10, 'AdaBoostRegressor'),
 (11, 'ExtraTreesRegressor'),
 (12, 'GradientBoostingRegressor'),
 (13, 'RandomForestRegressor')]

In [17]:
pred = np.mean(list(map(lambda x: regs[x][1].predict(test), [0, 12,13])), axis=0)
sub = pd.DataFrame({'Id':test['Id'], 'SalePrice':pred})
sub.to_csv('submission_Universe_fillNaN.csv', index=None)